# CRA Checks

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import yaml
import os
import pyodbc
import copy
import pandas as pd
import pyodbc
import os
import glob
import numpy as np
import math

In [2]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                    'Server=DDAMWSQL16.sandag.org;'
                    'Database=socioec_data;'
                    'Trusted_Connection=yes;')

In [4]:
dof_pop_query = '''SELECT *
  FROM [socioec_data].[ca_dof].[population_housing_estimates]
  WHERE area_name = 'San Diego' AND summary_type='Total' AND area_type = 'county' AND vintage_yr = 2021;'''

dof_pop =  pd.read_sql_query(dof_pop_query, conn) #example df

dof_pop

,area_type,area_name,summary_type,county_name,est_yr,est_md,total_pop,household_pop,group_quarters,total_hu,single,single_detached,single_attached,multiple,two_to_four,five_plus,mobile_homes,occupied,unoccupied,vintage_yr
0,County,San Diego,Total,San Diego,2017,JAN_1,3303366,3197067,106299,1201517,721664,616128,105536,433837,86281,347556,46016,1129464,72053,2021
1,County,San Diego,Total,San Diego,2010,JAN_1,3095313,2991515,103798,1164028,707604,603441,104163,410658,84621,326037,45766,1086113,77915,2021
2,County,San Diego,Total,San Diego,2014,JAN_1,3232762,3127529,105233,1180875,715522,610783,104739,419449,85494,333955,45904,1107562,73313,2021
3,County,San Diego,Total,San Diego,2020,JAN_1,3331279,3218869,112410,1226879,729293,622861,106432,451206,87814,363392,46380,1153981,72898,2021
4,County,San Diego,Total,San Diego,2011,JAN_1,3127603,3025703,101900,1166858,709313,605022,104291,411748,84620,327128,45797,1089420,77438,2021
5,County,San Diego,Total,San Diego,2018,JAN_1,3321118,3210345,110773,1210138,723430,617674,105756,440623,86682,353941,46085,1137791,72347,2021
6,County,San Diego,Total,San Diego,2015,JAN_1,3264706,3156628,108078,1188445,717461,612517,104944,425048,85792,339256,45936,1114909,73536,2021
7,County,San Diego,Total,San Diego,2021,JAN_1,3315404,3219058,96346,1234087,730771,624045,106726,456850,89426,367424,46466,1160772,73315,2021
8,County,San Diego,Total,San Diego,2012,JAN_1,3161808,3058316,103492,1170267,711359,606978,104381,413069,84884,328185,45839,1093511,76756,2021
9,County,San Diego,Total,San Diego,2016,JAN_1,3283009,3175539,107470,1193395,719600,614416,105184,427835,85921,341914,45960,1121242,72153,2021


In [3]:
dof_eth_query = '''SELECT [county_fips_code]
      ,[fiscal_yr]
      ,[race_code]
      ,[sex]
      ,[age]
      ,[population]
  FROM [socioec_data].[ca_dof].[population_proj_2021_07_14]
  WHERE county_fips_code = '06073';'''

dof_eth =  pd.read_sql_query(dof_eth_query, conn) #example df
dof_eth

,county_fips_code,fiscal_yr,race_code,sex,age,population
0,6073,2010,1,F,0,6877
1,6073,2010,1,F,1,6731
2,6073,2010,1,F,2,6743
3,6073,2010,1,F,3,6848
4,6073,2010,1,F,4,6651
...,...,...,...,...,...,...
79249,6073,2058,6,M,38,1116
79250,6073,2058,6,M,39,1081
79251,6073,2058,6,M,40,1004
79252,6073,2058,6,M,41,901


# Checking if the household size equation is correct

In [2]:
data = pd.read_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-58 2019 Base Year Forecast Output QC/data/MGRA15 Data/mgra15_based_input2019.csv')

In [13]:
sorted_data = data[['hhs', 'hhp', 'hh']]
sorted_data['Calulated_hhs'] = sorted_data['hhp'] / sorted_data['hh']
# sorted_data['Flag'] = ~(sorted_data['Calulated_hhs'] == sorted_data['hhs']) #True means there is a flag
# sorted_data[sorted_data['Flag'] == True].to_excel('C:/Users/cra/Downloads/edd_sandag_sector_diff.xlsx')
sorted_data[~(np.isclose(sorted_data['Calulated_hhs'], sorted_data['hhs'], rtol=.1))].dropna()

C:\Users\cra\AppData\Local\Temp\ipykernel_6080\1000657331.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_data['Calulated_hhs'] = sorted_data['hhp'] / sorted_data['hh']


,hhs,hhp,hh,Calulated_hhs
0,3.140,760.0,172.0,4.418605
8,7.213,968.0,243.0,3.983539
26,5.805,437.0,67.0,6.522388
30,2.858,603.0,186.0,3.241935
34,3.108,115.0,18.0,6.388889
...,...,...,...,...
24308,3.750,390.0,29.0,13.448276
24312,2.341,103.0,24.0,4.291667
24313,2.422,109.0,27.0,4.037037
24315,2.513,1141.0,226.0,5.048673


In [ ]:
# Some of the rollups do not have the correct information inside
comparison_processed_data_jur['vacancy_rate'] = ((comparison_processed_data_jur['vacancy'] - comparison_processed_data_jur['unoccupiable']) / comparison_processed_data_jur['units']) * 100

# Fixing HHS:  hhp / Household Population (hh) <- it will have the new value
comparison_processed_data_jur['hhs'] = comparison_processed_data_jur['hhp'] / comparison_processed_data_jur['hh']

In [11]:
sorted_data[~(np.isclose(sorted_data['Calulated_hhs'], sorted_data['hhs'], rtol=.1))].dropna().to_excel('C:/Users/cra/Downloads/edd_sandag_sector_diff.xlsx')

,hhs,hhp,hh,Calulated_hhs,Flag
0,3.140,760.0,172.0,4.418605,True
8,7.213,968.0,243.0,3.983539,True
26,5.805,437.0,67.0,6.522388,True
30,2.858,603.0,186.0,3.241935,True
34,3.108,115.0,18.0,6.388889,True
...,...,...,...,...,...
24308,3.750,390.0,29.0,13.448276,True
24312,2.341,103.0,24.0,4.291667,True
24313,2.422,109.0,27.0,4.037037,True
24315,2.513,1141.0,226.0,5.048673,True
